In [ ]:
import polars as pl
import pandas as pd

df = pl.read_excel(r"C:\Users\wallace.souza\Desktop\mix - fonseca - wanderley.xlsx")
est = pl.read_excel(r"C:\Users\wallace.souza\Desktop\estoque.xlsx")


In [151]:
df = df.with_columns(pl.col("Produto : Mês").str.split(":").list.get(1).alias('Ano'))
df = df.with_columns(pl.col("Ano").str.split("/").list.get(0).alias('Mês'))
df = df.with_columns(pl.col("Ano").str.split("/").list.get(1))

In [152]:
df

SEQPRODUTO,Produto : Mês,Venda Quantidade,Ano,Mês
i64,str,f64,str,str
940,"""ABACAXI SCHRAMM LT 400G RODELA…",6.0,"""2024""",""" JAN"""
940,"""ABACAXI SCHRAMM LT 400G RODELA…",4.0,"""2024""",""" FEV"""
940,"""ABACAXI SCHRAMM LT 400G RODELA…",4.0,"""2024""",""" MAR"""
940,"""ABACAXI SCHRAMM LT 400G RODELA…",5.0,"""2024""",""" ABR"""
940,"""ABACAXI SCHRAMM LT 400G RODELA…",2.0,"""2024""",""" MAI"""
…,…,…,…,…
5892,"""VINAGRE TTO UNICO 750ML : JUL/…",53.0,"""2025""",""" JUL"""
5892,"""VINAGRE TTO UNICO 750ML : AGO/…",56.0,"""2025""",""" AGO"""
5892,"""VINAGRE TTO UNICO 750ML : SET/…",43.0,"""2025""",""" SET"""


In [153]:
df = df.join(est, on='SEQPRODUTO', how='left')

In [154]:
df = df.filter((pl.col("QUANTIDADE ESTOQUE").is_not_null()) & (pl.col("QUANTIDADE ESTOQUE") != 0))

In [155]:
df1 = df.unique(subset=['SEQPRODUTO','Ano',"Mês"], keep='any')

In [156]:
df1_2024 = df1.filter(pl.col('Ano') == '2024')
df1_2025 = df1.filter(pl.col('Ano') == '2025')

In [157]:
df1_2024 = df1_2024.pivot(index='SEQPRODUTO',columns=['Ano', 'Mês'], values="Venda Quantidade")

C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_14716\771491037.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  df1_2024 = df1_2024.pivot(index='SEQPRODUTO',columns=['Ano', 'Mês'], values="Venda Quantidade")


In [158]:
df1_2025 = df1_2025.pivot(index='SEQPRODUTO',columns=['Ano', 'Mês'], values="Venda Quantidade")


C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_14716\1933558317.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  df1_2025 = df1_2025.pivot(index='SEQPRODUTO',columns=['Ano', 'Mês'], values="Venda Quantidade")


In [159]:
df1_2024 = df1_2024.join(df1_2025, on='SEQPRODUTO', how='outer')

C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_14716\3218417867.py:1: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  df1_2024 = df1_2024.join(df1_2025, on='SEQPRODUTO', how='outer')


In [160]:
df1_2024=df1_2024.with_columns(
    pl.when(pl.col("SEQPRODUTO").is_null())
      .then(pl.col("SEQPRODUTO_right"))
      .otherwise(pl.col("SEQPRODUTO"))
      .alias("SEQPRODUTO")
)


In [161]:
df1_2024=df1_2024.with_columns(
    pl.concat_list(['{"2024"," JAN"}',
 '{"2024"," NOV"}',
 '{"2024"," DEZ"}',
 '{"2024"," JUN"}',
 '{"2024"," SET"}',
 '{"2024"," FEV"}',
 '{"2024"," ABR"}',
 '{"2024"," OUT"}',
 '{"2024"," MAR"}',
 '{"2024"," MAI"}',
 '{"2024"," JUL"}',
 '{"2024"," AGO"}',
 '{"2025"," FEV"}',
 '{"2025"," JUN"}',
 '{"2025"," OUT"}',
 '{"2025"," JUL"}',
 '{"2025"," MAR"}',
 '{"2025"," ABR"}',
 '{"2025"," JAN"}',
 '{"2025"," MAI"}',
 '{"2025"," NOV"}',
 '{"2025"," SET"}',
 '{"2025"," AGO"}']).list.std().alias("std_linha")
)
